In [25]:
import os
import re

from kinnara.gather.twitter import TwitterApiWrapper

In [2]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s',
        level=logging.INFO)
logger = logging.getLogger(__name__)

#### read in api keys and access tokens

In [3]:
api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

access_token = os.getenv('ACCESS_TOKEN')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')

## Gather public figure tweets

In [4]:
# create kinnara gatherer
twitter_wrapper = TwitterApiWrapper(api_key=api_key, api_secret=api_secret,
                        access_token=access_token, access_token_secret=access_token_secret)

In [10]:
screen_names = [
    'BarackObama',
    'realDonaldTrump',
    'KimKardashian',
    'BillGates',
    'Oprah',
    'justinbieber',
    'Beyonce',
    'TheRock'
]

In [11]:
users = []
for screen_name in screen_names:
    users.append(twitter_wrapper.get_user(screen_name))

2018-08-22 10:41:39,245 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:39,417 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:39,607 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:39,782 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:39,968 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:40,147 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:40,340 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:40,508 - INFO - Starting new HTTPS connection (1): api.twitter.com


In [12]:
# lets grab 400 tweets from each twitter users timeline
for user in users:
    user['tweets'] = twitter_wrapper.get_tweets(user['id_str'], max_tweets_returned=400)

2018-08-22 10:41:47,348 - INFO - getting tweets for 813286
2018-08-22 10:41:47,353 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:47,911 - INFO - getting tweets for 813286
2018-08-22 10:41:47,915 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:48,260 - INFO - getting tweets for 25073877
2018-08-22 10:41:48,264 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:48,611 - INFO - getting tweets for 25073877
2018-08-22 10:41:48,614 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:48,966 - INFO - getting tweets for 25365536
2018-08-22 10:41:48,970 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:49,352 - INFO - getting tweets for 25365536
2018-08-22 10:41:49,356 - INFO - Starting new HTTPS connection (1): api.twitter.com
2018-08-22 10:41:49,989 - INFO - getting tweets for 50393960
2018-08-22 10:41:49,993 - INFO - Starting new HTTPS connection (1): api.t

In [54]:
def clean_tweet(tweet_text):
    '''clean up tweet text'''
    return re.sub(r' ?https[^ ]*', r'', tweet_text)

In [57]:
screen_name_to_tweets = {}
for user in users:
    screen_name_to_tweets[user['screen_name']] = [clean_tweet(tweet.get('full_text', ''))
                                                  for tweet in user['tweets']
                                                  if re.match(r'^RT @', tweet['full_text']) is None]

In [61]:
# save it for later
f = open('tweet_data.csv', 'w')
for screen_name, tweets in screen_name_to_tweets.items():
    for tweet in tweets:
        f.write(f'"{screen_name}", "{tweet}"\n')
f.close()

# Model